# Machine Learning for Demand Forecasting

### Use case - predicting the demand for items at a libary

In [93]:
# sfOptions = {
#   "sfURL" : "datalytyx.east-us-2.azure.snowflakecomputing.com",
#   "sfAccount" : "datalytyx",
#   "sfUser" : "WILLHOLTAM",
#   "sfPassword" : "04MucSfLV",
#   "sfRole": "DATABRICKS",
#   "sfDatabase" : "DATABRICKS_DEMO",
#   "sfSchema" : "SEATTLE_LIBRARY",
#   "sfWarehouse" : "DATASCIENCE_WH"
# }
# SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

In [94]:
# spark.conf.set("spark.executor.memory", "10g")
# spark.conf.set("spark.databricks.io.cache.enabled", "true")
# #spark.conf.set("spark.databricks.io.cache.maxDiskUsage", "10g")
# spark.conf.set("spark.databricks.io.cache.maxMetaDataCache", "10g")
# # =true
# spark.databricks.io.cache.maxDiskUsage 10g
# spark.databricks.io.cache.maxMetaDataCache 10g

# spark.conf.set(
#   "fs.azure.account.key.dlxhackathon1.blob.core.windows.net",
#   "ej8Nd4FyayPD9InmjCWZvVfB7nV4EWnkWu5IjWEUfs4aXRe7nz/xkPg6g1BtY6X5i+kHBQ8uDLGOG2PTpB28TQ==")
# df = spark.read.format("csv").option("inferSchema", "true").load("wasbs://hackathon@dlxhackathon1.blob.core.windows.net/")
# df.describe()

In [95]:
# Import Libraries
import numpy as np
import pandas as pd
import nltk  # Has to be added through Workspaces/ attach library to cluster
import more_itertools
import re
import os
import codecs
import mpld3
from snowflake.sqlalchemy import URL
from nltk.stem.snowball import SnowballStemmer
from sqlalchemy import create_engine
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import feature_extraction
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

In [96]:
sf_account = "datalytyx"
sf_user = "WILLHOLTAM"
sf_pwd = "04MucSfLV"
# sf_user = "CHRISSCHON"
# sf_pwd = "UpsetSheep7"
sf_role = "DATABRICKS"
sf_db = "DATABRICKS_DEMO"
sf_schema = "SEATTLE_LIBRARY"
sf_wh = "DATASCIENCE_WH"
sf_region = "east-us-2.azure"

In [97]:
engine = create_engine(URL(
    user = sf_user,
    password = sf_pwd,
    account = sf_account,
    region = sf_region,
    database = sf_db,
    schema = sf_schema,
    warehouse = sf_wh,
    role = sf_role,
))

# engine = create_engine(connection_string)

In [98]:
df_pandas = pd.read_sql_query("select * from library_collection_inventory where reportdate in ('2017-09-01T00:00:00','2017-10-01T00:00:00', '2017-11-01T00:00:00', '2017-12-01T00:00:00', '2018-01-01T00:00:00', '2018-01-01T00:00:00', '2018-02-01T00:00:00', '2018-02-01T00:00:00', '2018-03-01T00:00:00', '2018-04-01T00:00:00', '2018-05-01T00:00:00', '2018-06-01T00:00:00', '2018-07-01T00:00:00') limit 1000", engine)

In [99]:
# #spark.conf.set("spark.executor.cores",2)

# df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
#   .options(**sfOptions) \
#   .option("query", """select * from library_collection_inventory where reportdate in ('2017-09-01T00:00:00','2017-10-01T00:00:00', '2017-11-01T00:00:00', '2017-12-01T00:00:00', '2018-01-01T00:00:00', '2018-01-01T00:00:00', '2018-02-01T00:00:00', '2018-02-01T00:00:00', '2018-03-01T00:00:00', '2018-04-01T00:00:00', '2018-05-01T00:00:00', '2018-06-01T00:00:00', '2018-07-01T00:00:00') """) \
#   .load().limit(200000)

In [100]:
# # Create a view or table

# temp_table_name = "library_collection_inventory"

# df.createOrReplaceTempView(temp_table_name)

In [101]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\WilliamHoltam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\WilliamHoltam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [102]:
titles = df_pandas.loc[:, 'title'].values.tolist()
subjects = df_pandas.loc[:,'subjects'].values.tolist()
author = df_pandas.loc[:, 'author'].values.tolist()

In [103]:
subjects = [str(i) for i in subjects]
#subjects_c = ["" for line in subjects if line.strip()]

In [104]:
def xstr(s):
    if s is None:
        return ''
    return str(s)

In [105]:
subjects_c = []
for i in subjects:
  subjects_c.append(xstr(i))

In [106]:
# for i in subjects_c:
#   letters_only = re.sub("[^a-zA-Z]",  # Search for all non-letters
#                           " ",          # Replace all non-letters with spaces
#                           str(i))

In [107]:
print(titles[:10])

['Japanese American internment camps / William Dudley, book editor.', 'Guide to financial markets : why they exist and how they work / Marc Levinson.', 'The Garden of Eloquence : a rhetorical bestiary / Willard R. Espy ; drawings by Teresa Peekema Allen.', 'Dinosaur kisses / David Ezra Stein.', 'Career of evil / Robert Galbraith.', 'Sunspot changes and weather changes / by H.H. Clayton.', "What to feed your baby : a pediatrician's guide to the eleven essential foods to guarantee veggie-loving, no-fuss, healthy-eating kids / Tanya Altmann with Beth Saltz.", 'The craving mind : from cigarettes to smartphones to love - why we get hooked and how we can break bad habits / Judson Brewer ; foreword by Jon Kabat-Zinn.', 'Fahrenheit 451 / Ray Bradbury ; introduction by Neil Gaiman.', 'Mastering the art of French cooking. Volume two / by Julia Child and Simone Beck.']


In [108]:
print(subjects_c[:10])

['Japanese Americans Evacuation and relocation 1942 1945, World War 1939 1945 Concentration camps United States, World War 1939 1945 Japanese Americans', 'Finance', 'Figures of speech Dictionaries', 'Dinosaurs Juvenile fiction, Kissing Juvenile fiction, Humorous stories', 'Private investigators England London Fiction, Man woman relationships Fiction, Large type books, Mystery fiction, Suspense fiction', 'Sunspots, Weather forecasting', 'Infants Nutrition, Food habits', 'Habit, Habit breaking, Change Psychology, Behavior modification', 'State sponsored terrorism Fiction, Totalitarianism Fiction, Book burning Fiction, Censorship Fiction, Political fiction, Satirical literature, Science fiction', 'Cooking French']


In [109]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')
print(stopwords[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [110]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# text_clf = Pipeline([('vect', CountVectorizer()),
#                      ('tfidf', TfidfTransformer()),
#                      ('clf', MultinomialNB()),
#                     ])


In [111]:
subjects_c

['Japanese Americans Evacuation and relocation 1942 1945, World War 1939 1945 Concentration camps United States, World War 1939 1945 Japanese Americans',
 'Finance',
 'Figures of speech Dictionaries',
 'Dinosaurs Juvenile fiction, Kissing Juvenile fiction, Humorous stories',
 'Private investigators England London Fiction, Man woman relationships Fiction, Large type books, Mystery fiction, Suspense fiction',
 'Sunspots, Weather forecasting',
 'Infants Nutrition, Food habits',
 'Habit, Habit breaking, Change Psychology, Behavior modification',
 'State sponsored terrorism Fiction, Totalitarianism Fiction, Book burning Fiction, Censorship Fiction, Political fiction, Satirical literature, Science fiction',
 'Cooking French',
 'Tom Fictitious character Hanna and Barbera Juvenile fiction, Jerry Fictitious character Hanna and Barbera Juvenile fiction, Cats Juvenile fiction, Mice Juvenile fiction, Humorous stories',
 'Plantation life North Carolina Fiction, Large type books, North Carolina Fict

In [112]:
# Define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [113]:
#use extend so it's a big flat list of vocab
totalvocab_stemmed = []
totalvocab_tokenized = []

for i in subjects_c:
    allwords_stemmed = tokenize_and_stem(str(i)) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    
    allwords_tokenized = tokenize_only(str(i))
    totalvocab_tokenized.extend(allwords_tokenized)

In [114]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')

there are 13903 items in vocab_frame


In [115]:
print(vocab_frame.head())

               words
japanes     japanese
american   americans
evacu     evacuation
and              and
reloc     relocation


In [116]:
from sklearn.feature_extraction.text import TfidfVectorizer
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=20000,
                                 min_df=5, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(subjects_c) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

Wall time: 417 ms
(1000, 736)


In [117]:
terms = tfidf_vectorizer.get_feature_names()

print(terms)

['18th', '18th centuri', '19th', '19th centuri', '20th', '20th centuri', '20th centuri biographi', '21st', '21st centuri', 'abil', 'action', 'action adventur', 'action adventur film', 'adult', 'adult fiction', 'adventur', 'adventur adventur', 'adventur fiction', 'adventur film', 'africa', 'african', 'african american', 'age', 'alaska', 'alien', 'alien encount', 'america', 'american', 'american 20th', 'american 20th centuri', 'american biographi', 'american fiction', 'american juvenil', 'american juvenil fiction', 'american juvenil literatur', 'american poetri', 'anecdot', 'anim', 'anim fiction', 'anim film', 'anim juvenil', 'anim juvenil fiction', 'anim relationship', 'anim televis', 'anim televis program', 'architectur', 'armi', 'art', 'aspect', 'audiobook', 'author', 'author american', 'autobiographi', 'automobil', 'b', 'bear', 'bear juvenil', 'bear juvenil fiction', 'bedtim', 'bedtim juvenil', 'bedtim juvenil fiction', 'behavior', 'biograph', 'biograph film', 'biographi', 'biographi

In [118]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [121]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

print(clusters)

Wall time: 417 ms
[0, 0, 0, 3, 1, 0, 0, 0, 1, 0, 3, 1, 0, 1, 0, 2, 0, 3, 0, 2, 0, 0, 1, 0, 0, 2, 0, 3, 0, 2, 3, 0, 0, 0, 0, 3, 0, 0, 0, 2, 1, 3, 0, 0, 0, 1, 0, 0, 3, 1, 1, 1, 3, 0, 0, 1, 1, 3, 2, 2, 1, 1, 0, 1, 0, 0, 1, 0, 0, 4, 1, 0, 0, 0, 4, 1, 1, 3, 3, 0, 1, 1, 0, 2, 3, 2, 0, 0, 0, 3, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 3, 1, 0, 4, 0, 1, 0, 4, 3, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 0, 1, 0, 3, 0, 2, 0, 0, 3, 3, 0, 0, 1, 0, 1, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 2, 1, 2, 3, 0, 0, 0, 3, 0, 0, 0, 0, 1, 0, 4, 1, 0, 1, 0, 0, 0, 1, 0, 3, 0, 2, 3, 1, 0, 0, 0, 3, 1, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 3, 0, 2, 0, 0, 0, 2, 0, 3, 3, 3, 0, 3, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 4, 2, 1, 0, 1, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 1, 3, 1, 0, 0, 0, 3, 0, 0, 0, 2, 0, 2, 0, 0, 3, 0, 4, 1, 2, 0, 4, 3, 0, 1, 0, 0, 0, 0, 0, 4, 0, 1, 1, 0, 0, 0, 2, 3, 0, 3, 0, 

In [120]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

#joblib.dump(km,  'doc_cluster.pkl')

#km = joblib.load('doc_cluster.pkl')
#clusters = km.labels_.tolist()

AttributeError: 'Pipeline' object has no attribute 'labels_'

In [125]:
books = { 'title': titles, 'author': author, 'subjects': subjects_c, 'cluster': clusters }

frame = pd.DataFrame(books, index = [clusters] , columns = ['title', 'author', 'cluster'])
frame

,title,author,cluster
0,Japanese American internment camps / William D...,None,0
0,Guide to financial markets : why they exist an...,"Levinson, Marc",0
0,The Garden of Eloquence : a rhetorical bestiar...,"Espy, Willard R.",0
3,Dinosaur kisses / David Ezra Stein.,"Stein, David Ezra",3
1,Career of evil / Robert Galbraith.,"Galbraith, Robert",1
0,Sunspot changes and weather changes / by H.H. ...,"Clayton, Henry Helm, 1861-1946",0
0,What to feed your baby : a pediatrician's guid...,"Altmann, Tanya Remer",0
0,The craving mind : from cigarettes to smartpho...,"Brewer, Judson.",0
1,Fahrenheit 451 / Ray Bradbury ; introduction b...,"Bradbury, Ray, 1920-2012",1
0,Mastering the art of French cooking. Volume tw...,"Child, Julia",0


In [123]:
# frame.columns
frame.cluster.value_counts() #number of books per cluster (clusters from 0 to 4)

0    561
1    157
3    145
2     84
4     53
Name: cluster, dtype: int64

In [124]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
  print("Cluster %d words:" % i, end='')
    
  for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
    print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
#    print("Cluster %d titles:" % i, end='')
#    for title in frame.loc[i,'title'].values.tolist():
#        print(' %s,' % title, end='')

Top terms per cluster:

Cluster 0 words: b'states',

 b'films',

 b'united',

 b'united',

 b'drama',

 b'history',

Cluster 1 words: b'fiction',

 b'mystery',

 b'mystery',

 b'investigators',

 b'fiction',

 b'fiction',

Cluster 2 words: b'juvenile',

 b'literature',

 b'juvenile',

 b'poetry',

 b'biography',

 b'biography',

Cluster 3 words: b'juvenile',

 b'fiction',

 b'juvenile',

 b'stories',

 b'schools',

 b'stories',

Cluster 4 words: b'music',

 b'rock',

 b'rock',

 b'popular',

 b'popular',

 b'music',



C:\Users\WilliamHoltam\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':


In [ ]:
import os  # for os.path.basename

import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS

MDS()

# convert two components as we're plotting points in a two-dimensional plane
# "precomputed" because we provide a distance matrix
# we will also specify `random_state` so the plot is reproducible.
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)  # shape (n_components, n_samples)

xs, ys = pos[:, 0], pos[:, 1]